In [ ]:
# activate virtual environment
# pip install requests
# pip install pymysql
# pip install bs4
# pip install lxml


In [2]:
import requests
from bs4 import BeautifulSoup
import pymysql

In [3]:
base_url = 'https://turbo.az/autos'

In [4]:
response = requests.get(base_url)

In [5]:
response

<Response [200]>

In [6]:
soup = BeautifulSoup(response.content, 'lxml')

In [7]:
page_content = soup.find('div', class_='products-container')

In [8]:
all_ = page_content.find_all('div', class_='tz-container')

In [9]:
elanlar = all_[4]

In [10]:
elanlar

<div class="tz-container"><div class="products-title"><p class="products-title-amount">10000 elan</p><div class="products-title-sort"><select id="sort-variants" name="sort-variants"><option data-path="/autos?q%5Bsort%5D=created_at" value="created_at">Tarixə görə</option>
<option data-path="/autos?q%5Bsort%5D=price_asc" value="price_asc">Əvvəlcə ucuz</option>
<option data-path="/autos?q%5Bsort%5D=price_desc" value="price_desc">Əvvəlcə bahalı</option>
<option data-path="/autos?q%5Bsort%5D=mileage" value="mileage">Yürüş</option>
<option data-path="/autos?q%5Bsort%5D=reg_year" value="reg_year">Buraxılış ili</option></select></div></div><div class="products"><div class="products-i"><a class="products-i__link" href="/autos/6368708-lada-vaz-niva" target="_blank"></a><a class="js-bookmark-item-6368708" data-method="post" data-remote="true" href="/autos/6368708-lada-vaz-niva/bookmarks" rel="nofollow"><div class="bookmarking"></div></a><a class="hide js-unbookmark-item-6368708" data-method="dele

In [11]:
items = elanlar.find_all('div', class_='products-i')

In [12]:
len(items)

24

In [13]:
items[0]

<div class="products-i"><a class="products-i__link" href="/autos/6368708-lada-vaz-niva" target="_blank"></a><a class="js-bookmark-item-6368708" data-method="post" data-remote="true" href="/autos/6368708-lada-vaz-niva/bookmarks" rel="nofollow"><div class="bookmarking"></div></a><a class="hide js-unbookmark-item-6368708" data-method="delete" data-remote="true" href="/autos/6368708-lada-vaz-niva/bookmarks" rel="nofollow"><div class="bookmarking added"></div></a><div class="products-i__top"><img alt="LADA (VAZ) Niva" loading="lazy" src="https://turbo.azstatic.com/uploads/f460x343/2022%2F06%2F09%2F18%2F59%2F01%2F06858b4c-5af5-456a-b0ab-80a20d7353f1%2F85114_Zz7YXmUQgX0_U_O9wEIaNQ.jpg"/><div class="products-i__info"></div></div><div class="products-i__bottom"><div class="products-i__price products-i__bottom-text"><div class="product-price">8 500 <span>AZN</span></div></div><div class="products-i__name products-i__bottom-text">LADA (VAZ) Niva</div><div class="products-i__attributes products-i_

In [14]:
for item in items:
    item_url = 'https://turbo.az' + item.find('a').get('href')
    details = item.find('div', class_='products-i__bottom')

    marka_model = details.find('div', class_='products-i__name products-i__bottom-text').text
    price = details.find('div', class_='product-price').text
    
    
    
    print(item_url, marka_model, price)
    print('===' * 30)

https://turbo.az/autos/6368708-lada-vaz-niva LADA (VAZ) Niva 8 500 AZN
https://turbo.az/autos/6355461-opel-astra Opel Astra 12 700 AZN
https://turbo.az/autos/6368704-mercedes-cls-350 Mercedes CLS 350 19 000 AZN
https://turbo.az/autos/6368703-lada-vaz-2115 LADA (VAZ) 2115 7 000 AZN
https://turbo.az/autos/6312031-chevrolet-trax Chevrolet Trax 24 800 AZN
https://turbo.az/autos/6357254-kia-optima Kia Optima 31 500 AZN
https://turbo.az/autos/6368701-daewoo-gentra Daewoo Gentra 9 000 AZN
https://turbo.az/autos/6368700-opel-zafira Opel Zafira 9 800 AZN
https://turbo.az/autos/6368712-toyota-rav-4 Toyota RAV 4 32 300 $
https://turbo.az/autos/6368699-ford-fusion Ford Fusion 19 000 AZN
https://turbo.az/autos/6355765-mitsubishi-mirage Mitsubishi Mirage 12 900 AZN
https://turbo.az/autos/6324514-bmw-320 BMW 320 15 800 AZN
https://turbo.az/autos/6360229-opel-astra Opel Astra 12 900 AZN
https://turbo.az/autos/6368682-iveco-astra-hd8 Iveco Astra HD8 34 999 AZN
https://turbo.az/autos/6368695-tofas-sahin

In [15]:
connection = pymysql.connect(
    host='localhost', # 127.0.0.1
    user='root',
    password ='mysql123',
    database='turbo_db'
)

In [16]:
create_table = """
create table cars (
    id int(11) not null auto_increment,
    url varchar(400) not null,
    price varchar(100),
    marka_model varchar(200),
    primary key (id)
)
"""

In [17]:
with connection.cursor() as cursor:
    cursor.execute(create_table)

In [18]:
for item in items:
    item_url = 'https://turbo.az' + item.find('a').get('href')
    
    details = item.find('div', class_='products-i__bottom')

    marka_model = details.find('div', class_='products-i__name products-i__bottom-text').text
    price = details.find('div', class_='product-price').text
    
    insert_sql = 'insert into cars (url, price, marka_model) values(%s, %s, %s)'
    values = (item_url, price, marka_model)
    
    with connection.cursor() as cursor:
        cursor.execute(insert_sql, values)
        connection.commit()